In [1]:
import pandas as pd
import numpy as np
import requests, bs4
import re, os
import matplotlib.pyplot as plt
import statsapi
pd.options.display.max_columns = 999
import tweepy
from datetime import date


In [2]:
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''



auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [46]:
# use this cell to update Twitter Profile
api.update_profile(description=f"Who's on the hunt to catch Joe Dimaggio?  This account tracks all MLB Hit Streaks of more than 5 games")

User(_api=<tweepy.api.API object at 0x000001EA41354D00>, _json={'id': 1545772813994250240, 'id_str': '1545772813994250240', 'name': 'DiMaggio Tracker', 'screen_name': 'DiMaggioTracker', 'location': 'Holy Cross Catholic Cemetary', 'profile_location': None, 'description': "Who's on the hunt to catch Joe Dimaggio?  This account tracks all MLB Hit Streaks of 5 games or more", 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 6, 'friends_count': 0, 'listed_count': 0, 'created_at': 'Sat Jul 09 14:12:41 +0000 2022', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 54, 'lang': None, 'status': {'created_at': 'Sat Jul 16 21:03:28 +0000 2022', 'id': 1548413026520293381, 'id_str': '1548413026520293381', 'text': '#RingTheBell\n\nNick Castellanos hit a double✌ off of a 94.0 mph cutter from Max Meyer, and now has a 6 game hit stre… https://t.co/aopushSs44', 'truncated': True, 'entities

In [3]:
team_dict = {
    108: ('LAA', 'Angels', '#GoHalos'),
    109: ('ARI', 'D-backs', '#Dbacks'),
    110: ('BAL', 'Orioles', '#Birdland'),
    111: ('BOS', 'Red Sox', '#DirtyWater'),
    112: ('CHC', 'Cubs', '#ItsDifferentHere'),
    113: ('CIN', 'Reds', '#ATOBTTR'),
    114: ('CLE', 'Guardians', '#ForTheLand'),
    115: ('COL', 'Rockies', '#Rockies'),
    116: ('DET', 'Tigers', '#DetroitRoots'),
    117: ('HOU', 'Astros', '#LevelUp'),
    118: ('KC', 'Royals', '#TogetherRoyal'),
    119: ('LAD', 'Dodgers', '#AlwaysLA'),
    120: ('WSH', 'Nationals', '#NATITUDE'),
    121: ('NYM', 'Mets', '#LGM'),
    133: ('OAK', 'Athletics', '#DrumTogether'),
    134: ('PIT', 'Pirates', '#LetsGoBucs'),
    135: ('SD', 'Padres', '#TimeToShine'),
    136: ('SEA', 'Mariners', '#SeaUsRise'),
    137: ('SF', 'Giants', '#SFGameUp'),
    138: ('STL', 'Cardinals', '#STLCards'),
    139: ('TB', 'Rays', '#RaysUp'),
    140: ('TEX', 'Rangers', '#StraightUpTX'),
    141: ('TOR', 'Blue Jays', '#NextLevel'),
    142: ('MIN', 'Twins', '#MNTwins'),
    143: ('PHI', 'Phillies', '#RingTheBell'),
    144: ('ATL', 'Braves', '#ForTheA'),
    145: ('CWS', 'White Sox', '#ChangeTheGame'),
    146: ('MIA', 'Marlins', '#MakeItMiami'),
    147: ('NYY', 'Yankees', '#RepBX'),
    158: ('MIL', 'Brewers', '#ThisIsMyCrew')
}

In [4]:
today_schedule = statsapi.schedule()

# get today's game ids.  Index 0 will give you all ids, Index 1 will just return active game ids
def get_active_game_ids():
    all_game_ids = []
    active_game_ids = []
    
    for i in range(len(today_schedule)):
        if today_schedule[i]['status'] == 'In Progress':
            all_game_ids.append(today_schedule[i]['game_id'])
            active_game_ids.append(today_schedule[i]['game_id'])
        else: 
            all_game_ids.append(today_schedule[i]['game_id'])
    return all_game_ids, active_game_ids

get_active_game_ids()

([661064,
  663029,
  661054,
  661563,
  661716,
  663203,
  662813,
  662438,
  662653,
  661281,
  661079,
  662346,
  663314,
  662573,
  662184],
 [])

In [13]:
today_schedule

[{'game_id': 663046,
  'game_datetime': '2022-07-25T22:40:00Z',
  'game_date': '2022-07-25',
  'game_type': 'R',
  'status': 'In Progress',
  'away_name': 'Miami Marlins',
  'home_name': 'Cincinnati Reds',
  'away_id': 146,
  'home_id': 113,
  'doubleheader': 'N',
  'game_num': 1,
  'home_probable_pitcher': 'Nick Lodolo',
  'away_probable_pitcher': 'Trevor Rogers',
  'home_pitcher_note': '',
  'away_pitcher_note': '',
  'away_score': 1,
  'home_score': 0,
  'current_inning': 3,
  'inning_state': 'Bottom',
  'venue_id': 2602,
  'venue_name': 'Great American Ball Park',
  'summary': '2022-07-25 - Miami Marlins (1) @ Cincinnati Reds (0) (Bottom of the 3rd)'},
 {'game_id': 661055,
  'game_datetime': '2022-07-25T23:05:00Z',
  'game_date': '2022-07-25',
  'game_type': 'R',
  'status': 'In Progress',
  'away_name': 'Tampa Bay Rays',
  'home_name': 'Baltimore Orioles',
  'away_id': 139,
  'home_id': 110,
  'doubleheader': 'N',
  'game_num': 1,
  'home_probable_pitcher': 'Austin Voth',
  'away_

In [5]:
today = date.today()
today = str(today)[-2:]
today
url = f'https://www.baseballmusings.com/cgi-bin/CurStreak.py?EndDate=07%2F{today}%2F2022'
resp = requests.get(url)
with open('test_mlb_longesthitstreak', 'wb') as f:
    f.write(resp.content)
df_streaks = pd.read_html('test_mlb_longesthitstreak')
df_streaks = pd.DataFrame(df_streaks[1])
df_streaks = df_streaks.loc[(df_streaks['Player'] != 'Player')]
df_streaks.head()

,Player,Games,At Bats,Runs,Hits,HR,RBI,BB,K,BA,OBA,Slug%,Last Game Date
0,Austin Riley,17,69,15,30,8,15,4,15,.435,.466,.884,2022-07-25
1,Carson Kelly,13,45,11,19,3,9,3,7,.422,.458,.756,2022-07-25
2,Trea Turner,12,52,12,18,4,10,1,7,.346,.358,.635,2022-07-25
3,Isiah Kiner-Falefa,12,44,2,13,0,10,0,6,.295,.304,.364,2022-07-24
4,Alec Bohm,10,33,7,16,2,7,3,3,.485,.514,.727,2022-07-25


In [7]:
def get_game_id(team_id):
    for game in today_schedule:
        if team_id == game['away_id']:
            return game['game_id']
        elif team_id == game['home_id']:
            return game['game_id'] 
        else:
            return 'does not play today'

In [8]:
def is_active(game_id):
    active_game_ids = []
    for i in range(len(today_schedule)):
        if today_schedule[i]['status'] == 'In Progress':
            active_game_ids.append(today_schedule[i]['game_id'])
        else: 
            continue
    if game_id in active_game_ids:
        return True
    else:
        return False

In [ ]:
def get_todays_hitstreak_info(df):
    todays_info = []
    for name in df['Player']:
        print(name)
        team_id = statsapi.lookup_player(name)[0]['currentTeam']['id']
        full_team_name = statsapi.lookup_team(team_id)[0]['name']
        team_abbrv = statsapi.lookup_team(team_id)[0]['teamCode']
        team_name = statsapi.lookup_team(team_id)[0]['teamName']
        player_info = (name, full_team_name, team_id, team_abbrv.upper(), team_name)
        todays_info.append(player_info)
    today_df = pd.DataFrame(todays_info, columns = ['Player', 'Full Team Name', 'Team ID', 'Team Abbrv', 'Mascot']).sort_values(by = 'Full Team Name')
    #today_df['game_id'] = today_df['Team ID'].map(lambda x: get_game_id(x))
    #today_df['is_active'] = today_df['game_id'].map(lambda x: is_active(x))
    return today_df
        
get_todays_hitstreak_info(df_streaks)

Austin Riley
Carson Kelly
Trea Turner
Isiah Kiner-Falefa
Alec Bohm
Harold Castro
Alex Kirilloff
Steven Kwan


In [16]:
s = 'Jackie Bradley, Jr'

In [20]:
import string
s.translate(string.punctuation)

'Jackie Bradley, Jr'

In [9]:
todays_tweets = ["Michael Harris II", "Steven Kwan", "Harold Castro", "J.T. Realmuto", "Austin Riley", "Alec Bohm", "Ramon Urias", "Donovan Solano"]

In [10]:
game_ids = get_active_game_ids()[1]
for game in game_ids:
    url = f'https://statsapi.mlb.com/api/v1/game/{game}/playByPlay'
    res = requests.get(url)
    data = res.json()
    
    for i in range(len(data['allPlays'])-1):
        try: 
            event = data['allPlays'][i]['result']['event']
        except:
            continue
        
        # Continue only for hits
        if event == 'Single' or event == 'Double' or event == 'Triple' or event == 'Home Run':
            
            if event == 'Home Run':
                event_emoji = '\U0001F4A3\U0001F680'
            elif event == 'Single':
                event_emoji = '\U000026BE'
            elif event == 'Double':
                event_emoji = '\U0000270C'
            elif event == 'Triple':
                event_emoji = '\U0001F3C3\U0001F3C3\U0001F3C3' 
            
            # save name and last_name.  Suffixes like "Jr" will need to be added going forward
            name = data["allPlays"][i]["matchup"]["batter"]["fullName"]
                      
            if name not in todays_tweets:
                if 'Jr' in name:
                    last_name = ' '.join(name.split()[-2:])
                else:
                    last_name = name.split()[-1]
                
                # identify what team batter is on
                player_team_id = statsapi.lookup_player(name)[0]['currentTeam']['id']

                #check to see if player has a hitting streak of 5 or more games
                df = pd.read_html('test_mlb_longesthitstreak')
                df = pd.DataFrame(df[1])
                if name in df['Player'].value_counts():
                    hit_streak = int(df.loc[(df['Player']== name)]['Games'].values[0]) + 1
                    print(name, hit_streak)
                else:
                    continue

                if hit_streak >= 5:
                    # save details from the hit to variables
                    pitcher_name = data['allPlays'][i]['matchup']['pitcher']['fullName']
                    pitch_type = data['allPlays'][0]['playEvents'][-1]['details']['type']['description']
                    pitch_speed = data['allPlays'][0]['playEvents'][-1]['pitchData']['startSpeed']
                    exit_velo = data['allPlays'][i]['playEvents'][-1]['hitData']['launchSpeed']
                    
                    # set emoji for launch_angle
                    launch_angle = data['allPlays'][i]['playEvents'][-1]['hitData']['launchAngle']
                    if 25 <= launch_angle <= 35:
                        launch_angle_emoji = '\U0001F64C'
                    elif launch_angle < 10:
                        launch_angle_emoji = '\U0001F447'
                    else:
                        launch_angle_emoji = ''
                    
                    # set emoji for batting average 
                    batting_avg =  float(df.loc[(df['Player']== name)]['BA'].values[0])
                    
                    if .300 <= batting_avg < .400:
                        heat_check_emoji = '\U0001F440'
                    elif .400 <= batting_avg < .500:
                        heat_check_emoji = '\U0001F525'
                    elif batting_avg >= .500:
                        heat_check_emoji = '\U0001F60D'
                    else:
                        heat_check_emoji = ''
                    
                    # set emoji for exit velocity
                    if exit_velo > 110:
                        exit_velo_emoji = '\U00002757\U00002757'
                    elif exit_velo >= 100:
                        exit_velo_emoji = '\U000026A1'
                    elif exit_velo >= 90:
                        exit_velo_emoji = '\U0001F44F'
                    else:
                        exit_velo_emoji = ''
                    
                    # Fix batting average to be three significant digits                 
                    if len(str(batting_avg)) == 3:
                        avg_fixed = str(batting_avg) + '0'
                    elif len(str(batting_avg)) == 2:
                        avg_fixed = str(batting_avg) + '00'
                    else:
                        avg_fixed = batting_avg
                    avg_fixed = str(avg_fixed).lstrip('0')
                    
                    # Send Tweet
                    if event == 'Double':
                        upload_result = api.media_upload(r'C:\Users\aorfa\Downloads\Utah_Get_Me_Two.mp4')
                        text = f'{team_dict[player_team_id][-1]}\n\n{name} hit a {event.lower()}{event_emoji} off of a {pitch_speed} mph {pitch_type.lower()} from {pitcher_name}, and now has a {hit_streak} game hit streak.\n\nExit Velocity: {exit_velo} mph{exit_velo_emoji}\n\nLaunch Angle: {launch_angle} degrees{launch_angle_emoji}\n\n{last_name} is batting {avg_fixed} over this stretch.{heat_check_emoji}'
                        api.update_status(status = text, media_ids = [upload_result.media_id_string])
                        print(f'tweet fired for {name}')
                    else:
                        api.update_status(f'{team_dict[player_team_id][-1]}\n\n{name} hit a {event.lower()}{event_emoji} off of a {pitch_speed} mph {pitch_type.lower()} from {pitcher_name}, and now has a {hit_streak} game hit streak.\n\nExit Velocity: {exit_velo} mph{exit_velo_emoji}\n\nLaunch Angle: {launch_angle} degrees{launch_angle_emoji}\n\n{last_name} is batting {avg_fixed} over this stretch.{heat_check_emoji}')
                        print(f'tweet fired for {name}')                                                     
                    
                    # Add name to list so that another a hit from the player won't trigger a tweet.
                    todays_tweets.append(name)
                    break
                else:
                    print(f"{name}'s hit streak is less than 5")
                    continue
            else:
                print(f'{name} already tweeted about')

Steven Kwan already tweeted about
Andrew Benintendi 6
tweet fired for Andrew Benintendi
Elias Diaz 7
tweet fired for Elias Diaz
Jeremy Pena 7
tweet fired for Jeremy Pena


In [11]:
todays_tweets 

['Michael Harris II',
 'Steven Kwan',
 'Harold Castro',
 'J.T. Realmuto',
 'Austin Riley',
 'Alec Bohm',
 'Ramon Urias',
 'Donovan Solano',
 'Andrew Benintendi',
 'Elias Diaz',
 'Jeremy Pena']

In [50]:
data['allPlays'][i]['playEvents']

[{'details': {'call': {'code': 'B', 'description': 'Ball'},
   'description': 'Ball',
   'code': 'B',
   'ballColor': 'rgba(39, 161, 39, 1.0)',
   'trailColor': 'rgba(188, 0, 33, 1.0)',
   'isInPlay': False,
   'isStrike': False,
   'isBall': True,
   'type': {'code': 'FF', 'description': 'Four-Seam Fastball'},
   'hasReview': False},
  'count': {'balls': 1, 'strikes': 0, 'outs': 0},
  'pitchData': {'startSpeed': 94.2,
   'endSpeed': 86.8,
   'strikeZoneTop': 3.65,
   'strikeZoneBottom': 1.66,
   'coordinates': {'aY': 27.42,
    'aZ': -9.59,
    'pfxX': -4.4,
    'pfxZ': 11.76,
    'pX': 0.71,
    'pZ': 3.55,
    'vX0': 6.99,
    'vY0': -137.04,
    'vZ0': -5.7,
    'x': 90.03,
    'y': 143.01,
    'x0': -1.29,
    'y0': 50.0,
    'z0': 6.3,
    'aX': -8.46},
   'breaks': {'breakAngle': 20.4, 'spinRate': 1978, 'spinDirection': 200},
   'zone': 12,
   'plateTime': 0.4,
   'extension': 6.79},
  'index': 0,
  'playId': '9951180d-d209-48e0-acdd-7857cb49c91a',
  'pitchNumber': 1,
  'startTi

In [39]:
exit_velo = data['allPlays'][i]['playEvents'][0]['hitData']['launchSpeed']
launch_angle = data['allPlays'][i]['playEvents'][0]['hitData']['launchAngle']

In [83]:
# https://stackoverflow.com/questions/51106363/tweet-mp4-files-with-tweepy.  I did not pip install the forked tweepy and it still worked.

upload_result = api.media_upload(r'C:\Users\aorfa\Downloads\Utah_Get_Me_Two.mp4')
api.update_status(status = 'this GIF == a Double',media_ids = [upload_result.media_id_string])

In [119]:
len(str(batting_avg) + '0')

5

In [28]:
import toml
keys = toml.load('.keys_folder/secrets.toml')
# index into the data dictionary
keys['twitter_keys']['consumer_key']

''